# `Nuqleon.Json`

Provides a simple object model for JSON, a parser, a printer, and visitors.

> **Note:** The history of this assembly goes all the way back 2009, before `Newtonsoft.Json` became the de facto standard for JSON serialization on .NET. It also predates the `System.Text.Json` library in the BCL.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.Json.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.Json,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [1]:
System.Diagnostics.Debugger.Launch();

## The JSON object model

This library provides a straightforward object model for JSON consisting of `null`, Boolean values, Number values, String values, arrays, and objects. An example of constructing a JSON object using factory methods is shown below.

In [1]:
using Nuqleon.Json.Expressions;
using static Nuqleon.Json.Expressions.Expression;

var json =
    Object(new Dictionary<string, Expression>
    {
        {
            "Name",
            String("Bart")
        },
        {
            "Age",
            Number("21")
        },
        {
            "Hobbies",
            Array(
                String("Code"),
                String("Walk"),
                String("Run")
            )
        }
    });

JSON objects can be printed to produce a JSON string.

In [1]:
string s = json.ToString();

Console.WriteLine(s);

Alternatively, the `ToString` method has an overload that accepts a `StringBuilder` to append the JSON text to.

The default printing isn't pretty; it's compact. Using the `PrettyPrinter` we can create a more friendly representation.

In [1]:
var printer = new PrettyPrinter(indentationWidth: 2, indentationCharacter: ' ');

string pretty = printer.Visit(json);

Console.WriteLine(pretty);

It may come as a surprise to see that the pretty printer does not have a `Print` method; instead, we use a method called `Visit`. This is because the pretty printer is implemented as a visitor over the JSON object. We can write custom visitors as well.

In [1]:
class FindStringConstants : ExpressionVisitor
{
    public List<string> Strings { get; } = new List<string>();

    public override Expression VisitConstant(ConstantExpression c)
    {
        if (c.NodeType == ExpressionType.String)
        {
            Strings.Add((string)c.Value);
        }

        return c;
    }
}

var fsc = new FindStringConstants();

fsc.Visit(json);

Console.WriteLine(string.Join(", ", fsc.Strings));

The `PrettyPrinter` is implemented as an `ExpressionVisitor<string>` where each node is transformed into a string.

## Parsing JSON

Existing JSON fragments can also be parsed into the JSON object model representation using `Expression.Parse`. An example is shown below.

In [1]:
string json = "{\"bar\": 42, \"foo\": false, \"qux\": [ null, 3.14 ]}";

Expression expr = Expression.Parse(json);

To illustrate the different node types and their properties, we'll assert the structure of the resulting JSON object using a bunch of type checks. Note that Number values are represented with a `string`-based `Value` on `ConstantExpression`. This is done to ensure a full-fidelity representation of the nunmber rather than causing a conversion to any of the .NET types, which may be lossy.

In [1]:
bool test =
    expr is ObjectExpression o && o.Members.Count == 3 &&
        o.Members["bar"] is ConstantExpression bar && bar.NodeType == ExpressionType.Number &&
            bar.Value.Equals("42") &&
        o.Members["foo"] is ConstantExpression foo && foo.NodeType == ExpressionType.Boolean &&
            foo.Value.Equals(false) &&
        o.Members["qux"] is ArrayExpression qux && qux.ElementCount == 2 &&
            qux.Elements[0] is ConstantExpression qux0 && qux0.NodeType == ExpressionType.Null &&
            qux.Elements[1] is ConstantExpression qux1 && qux1.NodeType == ExpressionType.Number &&
                qux1.Value.Equals("3.14");

Console.WriteLine(test);

## JSON serialization

This library also contains a rudimentary JSON serializer and deserializer that goes back all the way to 2009. It is kept for compatibility but alternatives are strongly recommended for newer code. An example of its usage is shown below.

In [1]:
using Nuqleon.Json.Serialization;

record Person
{
    public string Name { get; init; }
    public int Age { get; init; }
}

var ser = new JsonSerializer(typeof(Person));

var oldBart = new Person { Name = "Bart", Age = 21 };

Console.WriteLine(oldBart);

string json = ser.Serialize(oldBart);

Console.WriteLine(json);

var newBart = (Person)ser.Deserialize(json);

Console.WriteLine(newBart);

Console.WriteLine(oldBart == newBart);